In [2]:
from app.dataprocessing.data_handler import DataHandler
from app.dataprocessing.benchmark import Timer, Plot, get_file_size_MB
from dotenv import load_dotenv
import os
import numpy as np
from app.dataprocessing.remote.opendap_access_cas import OpendapAccessCAS
import xarray as xr


In [3]:
load_dotenv()
USERNAME = os.environ.get("CMEMS_CAS_USERNAME")
PASSWORD = os.environ.get("CMEMS_CAS_PASSWORD")

cas_url = "https://cmems-cas.cls.fr/cas/login"

dataset_url = "https://my.cmems-du.eu/thredds/dodsC/dataset-reanalysis-nemo-dailymeans"
dataset_url = 'https://my.cmems-du.eu/thredds/dodsC/METOFFICE-GLO-SST-L4-REP-OBS-SST'
dataset_url = 'https://nrt.cmems-du.eu/thredds/dodsC/global-analysis-forecast-phy-001-024-monthly'
# ref : https://resources.marine.copernicus.eu/product-detail/BALTICSEA_REANALYSIS_PHY_003_011/INFORMATION

In [9]:
constraints = {'time' : 0, 'depth':0}
oa = OpendapAccessCAS(USERNAME, PASSWORD, dataset_url, cas_url, constraints=constraints)
oa.ds
print()
#print(len(dd.dims))
#oa.ds.isel(time=slice(14600, 14625)).to_netcdf('SST_GLO_SST_L4_REP_OBSERVATIONS_010_011_slice.nc')

<xarray.Dataset>
Dimensions:    (longitude: 4320, latitude: 2041)
Coordinates:
    depth      float32 0.494
  * longitude  (longitude) float32 -180.0 -179.9 -179.8 ... 179.8 179.8 179.9
  * latitude   (latitude) float32 -80.0 -79.92 -79.83 ... 89.83 89.92 90.0
    time       datetime64[ns] 2020-01-16T12:00:00
Data variables:
    mlotst     (latitude, longitude) float32 ...
    zos        (latitude, longitude) float32 ...
    bottomT    (latitude, longitude) float32 ...
    sithick    (latitude, longitude) float32 ...
    siconc     (latitude, longitude) float32 ...
    usi        (latitude, longitude) float32 ...
    vsi        (latitude, longitude) float32 ...
    thetao     (latitude, longitude) float32 ...
    so         (latitude, longitude) float32 ...
    uo         (latitude, longitude) float32 ...
    vo         (latitude, longitude) float32 ...
Attributes: (12/14)
    title:                         Monthly mean fields for product GLOBAL_ANA...
    references:                    http://marine.copernicus.eu
    credit:                        E.U. Copernicus Marine Service Information...
    licence:                       http://marine.copernicus.eu/services-portf...
    contact:                       servicedesk.cmems@mercator-ocean.eu
    producer:                      CMEMS - Global Monitoring and Forecasting ...
    ...                            ...
    area:                          GLOBAL
    product:                       GLOBAL_ANALYSIS_FORECAST_PHY_001_024
    dataset:                       global-analysis-forecast-phy-001-024-monthly
    source:                        MERCATOR PSY4QV3R1
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    quality_information_document:  http://marine.copernicus.eu/documents/QUID...

In [12]:
print('longitude' in oa.ds.dims)

True


In [50]:
oa = OpendapAccessCAS(USERNAME, PASSWORD, dataset_url, cas_url, 10)
import math
def estimate_file_size(d):
    d.isel(time=0).to_netcdf("slice.nc")
    file_size_bytes = os.path.getsize("slice.nc")
    file_size_MB = file_size_bytes / (1024 * 1024)

    time_dim = d.dims["time"]

    estimate_MB = time_dim * file_size_MB

    return int(math.ceil(estimate_MB))

print(estimate_file_size(oa.ds))
ds = xr.open_dataset('app/externalresources/datasets/BALTICSEA_REANALYSIS_PHY_003_011_slice.nc')
ds


1774360


<xarray.Dataset>
Dimensions:    (depth: 56, latitude: 523, longitude: 383, time: 30)
Coordinates:
  * depth      (depth) float32 1.501 4.513 7.54 10.58 ... 666.8 688.9 711.1
  * latitude   (latitude) float32 48.49 48.53 48.56 48.59 ... 65.82 65.86 65.89
  * longitude  (longitude) float32 9.014 9.069 9.125 9.18 ... 30.12 30.18 30.24
  * time       (time) datetime64[ns] 1993-01-01T12:00:00 ... 1993-01-30T12:00:00
Data variables:
    sob        (time, latitude, longitude) float32 ...
    bottomT    (time, latitude, longitude) float32 ...
    mlotst     (time, latitude, longitude) float32 ...
    so         (time, depth, latitude, longitude) float32 ...
    thetao     (time, depth, latitude, longitude) float32 ...
    uo         (time, depth, latitude, longitude) float32 ...
    vo         (time, depth, latitude, longitude) float32 ...
Attributes: (12/23)
    references:             http://www.smhi.se
    institution:            Swedish Meterological and Hydrological Institute
    history:                See source and creation_date attributees
    Conventions:            CF-1.5
    contact:                servicedesk_cmems@mercator-ocean.eu
    comment:                Provided by SMHI as a Copernicus Marine Environme...
    ...                     ...
    creation_date:          2021-11-09 UTC
    bullentin_date:         20201231
    start_date:             2020-12-31 UTC
    stop_date:              2020-12-31 UTC
    start_time:             00:00 UTC
    stop_time:              00:00 UTC

In [46]:
from functools import reduce

types = {
    "float32": 32,
    "float64": 64,
    "int32": 32,
    "int64": 64,
    "int16": 16,
    "int8": 8,
    "uint8": 8,
    "uint16": 16,
    "uint32": 32,
    "uint64": 64,
    "datetime64[ns]": 64,
    "datetime64[s]": 64,
    "datetime64[ms]": 64,
    "datetime64[us]": 64,
    "datetime64[m]": 64,
    "datetime64[D]": 64,
    "object": 64,
}

size_point = 0
for v in ds.data_vars:
    print(v)
    size_point += types[str(ds[v].dtype)]

dim_s = 0
for k in ds.dims:
    dim_s += types[str(ds[k].dtype)]
d_points = reduce(
    (lambda x, y: x * y),
    [ds.sizes.mapping[k] for k in ds.sizes.mapping],
)

byte_s = ((d_points) * size_point)/8
file_size_MB = byte_s / (1024 * 1024)
print(estimate_file_size(ds))
print(file_size_MB)

print(list(ds.dims.mapping.keys())[0])


sob
bottomT
mlotst
so
thetao
uo
vo
5205
8986.029968261719
depth


In [2]:
import xarray as xr
ds = xr.open_dataset('app/externalresources/datasets/BALTICSEA_REANALYSIS_PHY_003_011_slice.nc')


ds

<xarray.Dataset>
Dimensions:    (depth: 56, latitude: 523, longitude: 383, time: 30)
Coordinates:
  * depth      (depth) float32 1.501 4.513 7.54 10.58 ... 666.8 688.9 711.1
  * latitude   (latitude) float32 48.49 48.53 48.56 48.59 ... 65.82 65.86 65.89
  * longitude  (longitude) float32 9.014 9.069 9.125 9.18 ... 30.12 30.18 30.24
  * time       (time) datetime64[ns] 1993-01-01T12:00:00 ... 1993-01-30T12:00:00
Data variables:
    sob        (time, latitude, longitude) float32 ...
    bottomT    (time, latitude, longitude) float32 ...
    mlotst     (time, latitude, longitude) float32 ...
    so         (time, depth, latitude, longitude) float32 ...
    thetao     (time, depth, latitude, longitude) float32 ...
    uo         (time, depth, latitude, longitude) float32 ...
    vo         (time, depth, latitude, longitude) float32 ...
Attributes: (12/23)
    references:             http://www.smhi.se
    institution:            Swedish Meterological and Hydrological Institute
    history:                See source and creation_date attributees
    Conventions:            CF-1.5
    contact:                servicedesk_cmems@mercator-ocean.eu
    comment:                Provided by SMHI as a Copernicus Marine Environme...
    ...                     ...
    creation_date:          2021-11-09 UTC
    bullentin_date:         20201231
    start_date:             2020-12-31 UTC
    stop_date:              2020-12-31 UTC
    start_time:             00:00 UTC
    stop_time:              00:00 UTC

In [4]:
print(list(map(float,ds['latitude'])))

[48.49169921875, 48.52503204345703, 48.55836486816406, 48.591697692871094, 48.625030517578125, 48.658363342285156, 48.69169616699219, 48.72502899169922, 48.75836181640625, 48.79169464111328, 48.82502746582031, 48.858360290527344, 48.891693115234375, 48.925025939941406, 48.95835876464844, 48.99169158935547, 49.0250244140625, 49.05835723876953, 49.09169006347656, 49.125022888183594, 49.158355712890625, 49.191688537597656, 49.22502136230469, 49.25835418701172, 49.29168701171875, 49.32501983642578, 49.35835266113281, 49.391685485839844, 49.425018310546875, 49.458351135253906, 49.49168395996094, 49.52501678466797, 49.558349609375, 49.59168243408203, 49.62501525878906, 49.658348083496094, 49.691680908203125, 49.725013732910156, 49.75834655761719, 49.79167938232422, 49.82501220703125, 49.85834503173828, 49.89167785644531, 49.925010681152344, 49.958343505859375, 49.991676330566406, 50.02500915527344, 50.05834197998047, 50.0916748046875, 50.12500762939453, 50.15834045410156, 50.191673278808594,

In [7]:
for d in ds.dims:
    print(d)
    print(ds[d])
    print(ds[d].values.max())

depth
<xarray.DataArray 'depth' ()>
array(13329.2197926)
711.0592
latitude
<xarray.DataArray 'latitude' ()>
array(29911.18527222)
Coordinates:
    latitude  float32 65.89
65.89142
longitude
<xarray.DataArray 'longitude' ()>
array(7516.2831068)
Coordinates:
    longitude  float32 30.24
30.235765
time


UFuncTypeError: ufunc 'add' cannot use operands with types dtype('<M8[ns]') and dtype('<M8[ns]')

In [34]:
from app.datastructures.datastructure_interface import get_bounds, get_geo_bounds
bnd = get_bounds(ds)
for b in bnd:
    print(b)
print(type(get_bounds(ds)['depth'][0]))

print(get_geo_bounds(ds))

depth
latitude
longitude
time
<class 'numpy.float32'>
{'latitude': (48.49169921875, 65.89141845703125), 'longitude': (9.013754844665527, 30.23576545715332)}
